In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

import time
import datetime
import schedule

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [3]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "6g") \
    .appName("TwitterStreamApp") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

21/11/01 20:45:00 WARN Utils: Your hostname, hongyeobui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.219 instead (on interface en0)
21/11/01 20:45:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/hongyeob/.pyenv/versions/3.8.12/envs/.venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/hongyeob/.ivy2/cache
The jars for the packages stored in: /Users/hongyeob/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c32ca48e-c3b4-483f-87e6-8d5da88667c3;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in c

In [4]:
tweet_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "tweets") \
  .option("startingOffsets", "latest") \
  .load()

tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")

In [5]:
tweets_tab = tweet_df_string.withColumn('word', explode(split(col('value'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False). \
    filter(col('word').contains('#'))

In [6]:
writeTweet = tweets_tab.writeStream. \
    outputMode("complete"). \
    format("memory"). \
    queryName("tweetquery"). \
    trigger(processingTime='2 seconds'). \
    start()

In [7]:
def showTopTenHashTags():
    query = spark.sql("select * from tweetquery limit 10")
    print("Top 10 HashTags")
    print(f"{datetime.datetime.now().strftime('%y/%m/%d %H:%M:%S')}")
    query.show()

In [8]:
# 1분마다 Top 10 Trending HashTags 출력
schedule.every(1).minutes.do(showTopTenHashTags)

while True:
    schedule.run_pending()
    time.sleep(1)

Top 10 HashTags
21/11/01 20:46:08
+-------------------+-----+
|               word|count|
+-------------------+-----+
|[#HBD_TREASUREMAKER|    2|
|                #RT|    2|
|           #트레저…|    2|
|    💌\n\n#TREASURE|    2|
|     #halloween2021|    2|
|              #승관|    2|
|              #런쥔|    1|
|            #セブチ|    1|
|              #태용|    1|
|        \n\n📌#동해|    1|
+-------------------+-----+



KeyboardInterrupt: 